<a href="https://colab.research.google.com/github/valeromora/SenalesSistemasVM/blob/main/3_SerieyTransformadaFourier/Shazam2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalar librerias necesarias para descargar audios youtube

!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile #instalar sondfile

import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import yt_dlp as youtube_dl
import subprocess
import os
import soundfile as sf # para instalar pip install soundfile
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist

salsa_vs_rock="salsa_vs_rock"
#cargar modelo
#File name: 2023_10_19_22_06_19modelo
s_vs_r = joblib.load(salsa_vs_rock+".pkl")
s_vs_r.keys()
Xw_ = s_vs_r["Xw_"]
label = s_vs_r["label"]



#funcion para descargar mp3 desde youtube
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 8.0 MB/s eta 0:00:00
  Created w

FileNotFoundError: ignored

In [ ]:
#https://www.youtube.com/watch?v=alJgdP95MS4
evaluar = str(input("Ingrese el link de la canción: ")) #cancion para evaluar genero
download_ytvid_as_mp3(evaluar,"prueba") # se utiliza la funcion para descargar la cancion de youtube

prueba = "prueba"
subprocess.call(['ffmpeg','-y', '-i', prueba+'.mp3',
                   prueba+'.wav'])

#leer la cancion
nombre_out = "output.wav"
x, fs = sf.read(prueba+'.wav')#x es la canción
ts = 5 #t segmento
tp = np.array([20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180])#cantidad y ubicacion de segmentos
NS = len(tp)
x_p = np.zeros((NS,int(ts*fs),2))
i=0
prueba_c=[]
for ti in tp: #segmentos de tiempo
        x_p[i] = x[int(fs*ti):int(fs*(ti+ts)),:]
        #label[i] = int(prueba[-5]) #tipo de genero
        prueba_c += [prueba[:-6]]
        print(f"{i} lectura: {prueba}; segundo {ti}:{ti+ts}; ")
        i+=1


#calculo de fourier
vfp = np.fft.rfftfreq(x_p.shape[1],1/fs) #calculo vector de frecuencias
Xwp = np.fft.rfft(x_p,axis=1).mean(axis=-1) #transformada rapida de Fourier para señal Real a lo largo del tiempo (axis=1) y se promedian los dos canales
#normalizacion entre  0 y 1
scap = MinMaxScaler()
Xwp_ = scap.fit_transform(abs(Xwp).T).T#datos para comparar con la base


In [ ]:
p=cdist(Xwp_,Xw_)
g1 = 0
g2 = 0

for i in range(NS):
  ind = np.argmin(p[i,])
  g = label[ind]# lista de generos
  if g == 1:
    g1 += 1
  else:
    g2 += 1


if g1 > g2:
  print("El genero de la canción es rock")
elif g1 < g2:
  print("El genero de la canción es salsa")
else:
  print("Error")

In [ ]:
g1

In [ ]:
from IPython.display import Audio #reproducir segmento
i = 5
Audio(x_p[i].T,rate=fs)